In [4]:
#for the dividend and dividend rate
import blpapi
import datetime
from datetime import date

# https://data.bloomberglp.com/labs/sites/2/2014/07/blpapi-developers-guide-2.54.pdf

def data_request(x):
    ref_data_service = '//blp/refdata'

    securities = [x + "Equity"]
    fields = ['BDVD_PR_EX_DATES_AND_DVD_AMOUNTS','last_price']
    request_parameters = []
    overrides = []

    # Initialize a default SessionOptions (localhost:8194)
    sessionOptions = blpapi.SessionOptions()

    # Initialize a Session
    session = blpapi.Session(sessionOptions)

    if not session.start():
        return
    try:      
        # Open the service from which we will request historical data
        if not session.openService(ref_data_service):
            return

        # Retrieve previously opened service
        refDataService = session.getService(ref_data_service)

        # Reference data
        request = refDataService.createRequest('ReferenceDataRequest')
        
        # Fill the request parameters
        securities_element = request.getElement('securities')
        for security in securities:
            securities_element.appendValue(security)
        
        fields_element = request.getElement('fields')
        for field in fields:
            fields_element.appendValue(field)

        for element, element_value in request_parameters:
            request.set(element, element_value)

        overrides_element = request.getElement('overrides')
        for field_id, value in overrides:
            end_date_override = overrides_element.appendElement()
            end_date_override.setElement('fieldId', field_id)
            end_date_override.setElement('value', value)

        session.sendRequest(request)

        # Process received events   
        pool = True
        while(pool):
            # We provide timeout to give the chance for Ctrl+C handling:
            event = session.nextEvent(50)

            if event.eventType() == blpapi.Event.RESPONSE or event.eventType() == blpapi.Event.PARTIAL_RESPONSE:                
                partial_result = dvd_hist_all_response_handler(event)
                
                Dividend= partial_result[0]
                Dividend_rate= partial_result[1]
                #******change this to string again because the accepted format in javascript is string format only
                Spot_Price = float(partial_result[2])
                print(Dividend)
                print(Dividend_rate)
                print(Spot_Price)
                # Response completly received
                if event.eventType() == blpapi.Event.RESPONSE:
                    pool = False
            else:
                # Monotoring messages - They can be printed/logged
                continue
    finally:
        session.stop()


def dvd_hist_all_response_handler(event):
    
    dividend_date_result,Dividend_rate_result,Last_price,Full_div_data = [],[],[],[]
    
    for message in event:
        securitydata_array = message.getElement('securityData')
        for i in range(securitydata_array.numValues()):
            security_data = securitydata_array.getValueAsElement(i)
        
            security = security_data.getElementAsString('security')
            sequence_number = security_data.getElementAsInteger('sequenceNumber')
        
            if security_data.hasElement('securityError'):
                security_error = security_data.getElement('securityError')
                continue

            field_data = security_data.getElement('fieldData')
            
            dvd_hist_all_array = field_data.getElement('BDVD_PR_EX_DATES_AND_DVD_AMOUNTS')
            
            Last_price = field_data.getElementAsString('last_price')

            for j in range(dvd_hist_all_array.numValues()):  
                    
                dvd_hist_all = dvd_hist_all_array.getValueAsElement(j)                

                ex_date_element = dvd_hist_all.getElement('Ex-Date')
                
                Div_date= ex_date_element.getValueAsString() if not ex_date_element.isNull() else ''
                
                dt = datetime.datetime.strptime(Div_date, '%Y-%m-%d')
                #converting thr date format y-m-d date format to d-m-y format
                dividend_date_result.append('{0}-{1}-{2:02}'.format(dt.day, dt.month, dt.year ))

                Dividend_rate_result.append(dvd_hist_all.getElementAsString('Dividend Per Share'))
                
                Full_div_data=[dividend_date_result,Dividend_rate_result,Last_price]
                
    return Full_div_data



          
#Dividend= partial_result=['5-22-2023', '5-27-2024', '5-26-2025', '5-25-2026', '5-24-2027', '5-22-2028', '5-21-2029', '5-20-2030', '5-19-2031']
#Dividend_rate= partial_result= [3.75, 3.85, 4.0, 4.15, 4.25, 4.3, 4.44, 4.51, 4.58]
#spot=41.875

In [5]:
data_request('HSBA LN')

['23-2-2023', '17-8-2023', '14-3-2024', '15-8-2024', '13-3-2025', '14-8-2025', '12-3-2026', '13-8-2026', '11-3-2027', '12-8-2027', '9-3-2028', '17-8-2028', '15-3-2029', '16-8-2029', '14-3-2030', '15-8-2030', '13-3-2031', '14-8-2031', '11-3-2032', '12-8-2032']
['0.260000', '0.100000', '0.250000', '0.140000', '0.180000', '0.300000', '0.200000', '0.330000', '0.220000', '0.360000', '0.220000', '0.360000', '0.220000', '0.360000', '0.220000', '0.360000', '0.220000', '0.360000', '0.220000', '0.360000']
488.3


In [187]:
#for the BID, ASK and VOL PRICES

import blpapi

# https://data.bloomberglp.com/labs/sites/2/2014/07/blpapi-developers-guide-2.54.pdf

def Bloom_bid_ask_api(x):
    ref_data_service = '//blp/refdata'

    securities = [x + "Equity"]
    fields = ['BID','ASK','IVOL_MID']
    request_parameters = []
    overrides = []

    # Initialize a default SessionOptions (localhost:8194)
    sessionOptions = blpapi.SessionOptions()

    # Initialize a Session
    session = blpapi.Session(sessionOptions)

    if not session.start():
        return
    try:      
        # Open the service from which we will request historical data
        if not session.openService(ref_data_service):
            return

        # Retrieve previously opened service
        refDataService = session.getService(ref_data_service)

        # Reference data
        request_data = refDataService.createRequest('ReferenceDataRequest')
        
        # Fill the request parameters
        securities_element = request_data.getElement('securities')
        for security in securities:
            securities_element.appendValue(security)
        
        fields_element = request_data.getElement('fields')
        for field in fields:
            fields_element.appendValue(field)

        for element, element_value in request_parameters:
            request_data.set(element, element_value)

        overrides_element = request_data.getElement('overrides')
        for field_id, value in overrides:
            end_date_override = overrides_element.appendElement()
            end_date_override.setElement('fieldId', field_id)
            end_date_override.setElement('value', value)

        session.sendRequest(request_data)

        # Process received events   
        pool = True
        while(pool):
            # We provide timeout to give the chance for Ctrl+C handling:
            event = session.nextEvent(500)

            if event.eventType() == blpapi.Event.RESPONSE or event.eventType() == blpapi.Event.PARTIAL_RESPONSE:                
                partial_result = bid_ask_response_handler(event)
                #Mapping the lis to convert the list of string to list of float values
                Float_result=(list(map(float,partial_result)))
                
                Bid_price= Float_result[0]
                       
                ASK_Price= Float_result[1]
                
                Vol_value= "{0:.3f}".format(Float_result[2]/100)
                
                print(Bid_price,ASK_Price, Vol_value)
                # Response completly received
                if event.eventType() == blpapi.Event.RESPONSE:
                    pool = False
            else:
                # Monotoring messages - They can be printed/logged
                continue
    finally:
        session.stop()
    Output={ 'Bid_price': Bid_price,'ASK_Price': ASK_Price, 'Vol_value':Vol_value }
    return  Output
    
def bid_ask_response_handler(event):

    BID_ASK_Vol_values=[]
    
    for message in event:
        securitydata_array = message.getElement('securityData')
        for i in range(securitydata_array.numValues()):
            security_data = securitydata_array.getValueAsElement(i)
        
            security = security_data.getElementAsString('security')
            sequence_number = security_data.getElementAsInteger('sequenceNumber')
        
            if security_data.hasElement('securityError'):
                security_error = security_data.getElement('securityError')
                continue

            field_data = security_data.getElement('fieldData')
            
            BID_array = field_data.getElementAsString('BID')
            
            BID_ASK_Vol_values.append(BID_array)
            
            ASK_array = field_data.getElementAsString('ASK')

            BID_ASK_Vol_values.append(ASK_array)
            
            Vol_data = field_data.getElementAsString('IVOL_MID')
            
            BID_ASK_Vol_values.append(Vol_data)
            
    return BID_ASK_Vol_values

In [188]:
Bloom_bid_ask_api('BNP GR 11/18/22 C49 Equity')

# FILTERING THE SUB STRING TO GET MATCHED STRIKES IN BLOOM API

In [87]:
#  blp.bds("EUSA2 BGN Curncy", 'MID')
from datetime import datetime
import datetime
import pdblp
con = pdblp.BCon()
con.start()


#for searching the list of strikes
data=con.bulkref('BNP FP Equity',flds=['OPT_CHAIN'])

data=data.loc[:,"value"]

Ticker_data=[]

for i in data:
    Ticker_data.append(i)

sub_string='12/16/22 C50 Equity'

NEW_DATA= ([s for s in Ticker_data if sub_string in s])

NEW_DATA.insert(0,'Select Strike')

print(NEW_DATA)

[['1 WK', 7, 1.4028], ['2 WK', 14, 1.4035], ['1 MO', 30, 1.5167], ['2 MO', 60, 1.7478], ['3 MO', 90, 1.9202], ['6 MO', 180, 2.278], ['9 MO', 270, 2.5039], ['12 MO', 360, 2.628], ['18 MO', 540, 2.686], ['2 YR', 720, 2.668], ['3 YR', 1080, 2.5739]]
23
1.5167
['Select Strike', 'BN1 FP 12/16/22 C50 Equity', 'BN3 FP 12/16/22 C50 Equity', 'BN4 FP 12/16/22 C50 Equity', 'BNP GR 12/16/22 C50 Equity', 'BNPE GR 12/16/22 C50 Equity']


# SUBTRACTING TWO DATES TO FIND THE INTEREST RATE

In [100]:
strike_api_data=[{"Maturity_data":"12/16/22","Interest_Rate":[["1 WK",7,1.4028],["2 WK",14,1.4035],["1 MO",30,1.5167],["2 MO",60,1.7478],["3 MO",90,1.9202],["6 MO",180,2.278],["9 MO",270,2.5039],["12 MO",360,2.628],["18 MO",540,2.686],["2 YR",720,2.668],["3 YR",1080,2.5739]]}]
Strike_data, ticker_data, Mat_date, Interest_data=[],[],[],[]

for i in strike_api_data:
    Mat_date.append(i['Maturity_data'])
    Interest_data=(i['Interest_Rate'])
    
print('Maturity date',Mat_date)
print('Interest data', Interest_data)  

dt = str(date.today())
result = abs(datetime.datetime.strptime(dt, "%Y-%m-%d") - datetime.datetime.strptime(Mat_date[0], "%m/%d/%y")).days
print('Diff in dates', result)

def closest(Interest_data, result):
    return Interest_data[min(range(len(Interest_data)), key = lambda i: abs(Interest_data[i][1]-res))]
      
Interest_value=closest(Interest_data, result) 
 
New_Interest_value=Interest_value[2]

print(New_Interest_value)

Maturity date ['12/16/22']
Interest data [['1 WK', 7, 1.4028], ['2 WK', 14, 1.4035], ['1 MO', 30, 1.5167], ['2 MO', 60, 1.7478], ['3 MO', 90, 1.9202], ['6 MO', 180, 2.278], ['9 MO', 270, 2.5039], ['12 MO', 360, 2.628], ['18 MO', 540, 2.686], ['2 YR', 720, 2.668], ['3 YR', 1080, 2.5739]]
Diff in dates 23
1.5167


In [ ]:
#for the BID, ASK and VOL PRICES using stike value
import pdblp
con = pdblp.BCon()
con.start()

strike_api_data = 'BNP GR 11/18/22 C51 Equity'

data=con.ref(strike_api_data ,flds=['BID','ASK','IVOL_MID'])
print(data)
data=data.loc[:,"value"]

data[2]="{0:.3f}".format(data[2]/100)

Ticker_data=[]

for i in data:
    Ticker_data.append(i)
    
Output={ 'Bid_price': Ticker_data[0],'ASK_Price': Ticker_data[1], 'Vol_value':Ticker_data[2] }

print(Output)

In [ ]:
#for the MARKET-SPOT, BID, ASK and VOL PRICES
import json
import pdblp

con = pdblp.BCon(timeout=2000)
con.start()

Refresh_Data= [{"strikes_list":"BN4 FP 11/18/22 C52 Equity","ticker_name":"BNP FP"},
 {"strikes_list":"BN1 FP 11/18/22 P51 Equity","ticker_name":"BNP FP"},
 {"strikes_list":"BNP GR 11/18/22 P52 Equity","ticker_name":"BNP FP"}]
    
Strike_data, Ticker_data=[],[]

for i in Refresh_Data:
    Strike_data.append(i['strikes_list'])
    Ticker_data.append(i['ticker_name'])    
    
Market_spot=con.ref(Ticker_data[0] + "Equity",flds=['last_price'])
Market_spot=Market_spot.loc[:,"value"]


Strikes_combined_data=[]

for i in range(len(Strike_data)):
    Strikes_list_data=[]
    data=con.ref(Strike_data[i], flds=['IVOL_MID','BID','ASK'])
    data=data.loc[:,"value"]
    volatility=float(data.iloc[0]/100)
    Bid_price=data.iloc[1]
    Ask_price=data.iloc[2]
    Strikes_list_data.append("{0:.3f}".format(volatility))
    Strikes_list_data.append(Bid_price)
    Strikes_list_data.append(Ask_price)
    Strikes_combined_data.append(Strikes_list_data)


Output={ 'Strike_price_value': Market_spot[0],'Option_market_values': Strikes_combined_data }

print(Output)

# Currency

In [ ]:
from xbbg import blp

Ticker_value = blp.bdp(['BNP FP Equity'],flds=['CRNCY'])

Ticker_value=Ticker_value.loc[:,"crncy"]

Search_Bloom_ticker
for i in Ticker_value:
    Search_Bloom_ticker=i
    
print(Search_Bloom_ticker)

# Interest rates

In [40]:
#To get the Currency rates

from xbbg import blp
import pdblp
import numbers
con = pdblp.BCon(timeout=2000)
con.start()

strike_api_data = 'EESWE1Z BGN Curncy'

data=con.ref(strike_api_data ,flds=["PX_LAST"])

print(data)

#To get the entire currency rates from a yield curve
from xbbg import blp
data=[]
blp.bds("YCSW0045 INDEX", "CURVE_TENOR_RATES")   


               ticker    field   value
0  EESWE1Z BGN Curncy  PX_LAST  1.4028


,tenor,tenor_ticker,ask_yield,mid_yield,bid_yield,last_update
YCSW0045 INDEX,6M,EUR006M Index,2.338,2.338,2.338,2022-11-23
YCSW0045 INDEX,7M,EUFR0AG BGN Curncy,2.654,2.644,2.634,2022-11-23
YCSW0045 INDEX,8M,EUFR0BH BGN Curncy,2.823,2.811,2.799,2022-11-23
YCSW0045 INDEX,9M,EUFR0CI BGN Curncy,2.968,2.957,2.946,2022-11-23
YCSW0045 INDEX,10M,EUFR0DJ BGN Curncy,3.055,3.044,3.033,2022-11-23
YCSW0045 INDEX,11M,EUFR0EK BGN Curncy,3.096,3.085,3.074,2022-11-23
YCSW0045 INDEX,12M,EUFR0F1 BGN Curncy,3.129,3.119,3.109,2022-11-23
YCSW0045 INDEX,15M,EUFR0I1C BGN Curncy,3.113,3.102,3.093,2022-11-23
YCSW0045 INDEX,18M,EUFR011F BGN Curncy,3.022,3.010,3.002,2022-11-23
YCSW0045 INDEX,2Y,EUSA2 BGN Curncy,2.892,2.880,2.868,2022-11-23


In [39]:
#To get the multiple Currency rates using the Ticker

#EURO_Curncy= ['EESWE1Z BGN Curncy', 'EESWE2Z BGN Curncy', 'EESWEA BGN Curncy', 'EESWEB BGN Curncy', 'EESWEC BGN Curncy', 'EESWEF BGN Curncy', 'EESWEI BGN Curncy', 'EESWE1 BGN Curncy', 'EESWE1F BGN Curncy', 'EESWE2 BGN Curncy', 'EESWE3 BGN Curncy', 'EESWE4 BGN Curncy', 'EESWE5 BGN Curncy', 'EESWE6 BGN Curncy', 'EESWE7 BGN Curncy', 'EESWE8 BGN Curncy', 'EESWE9 BGN Curncy', 'EESWE10 BGN Curncy', 'EESWE12 BGN Curncy', 'EESWE15 BGN Curncy', 'EESWE20 BGN Curncy', 'EESWE30 BGN Curncy', 'EESWE40 BGN Curncy', 'EESWE50 BGN Curncy']
#EURO_tenor =['1 WK', '2 WK', '1 MO', '2 MO', '3 MO', '6 MO', '9 MO', '12 MO', '18 MO', '2 YR', '3 YR', '4 YR', '5 YR', '6 YR', '7 YR', '8 YR', '9 YR', '10 YR', '12 YR', '15 YR', '20 YR', '30 YR', '40 YR', '50 YR']

#GBP_Curncy = ['BPSWS1Z BGN Curncy', 'BPSWS2Z BGN Curncy', 'BPSWSA BGN Curncy', 'BPSWSB BGN Curncy', 'BPSWSC BGN Curncy', 'BPSWSD BGN Curncy', 'BPSWSE BGN Curncy', 'BPSWSG BGN Curncy', 'BPSWSH BGN Curncy', 'BPSWSI BGN Curncy', 'BPSWSJ BGN Curncy', 'BPSWSK BGN Curncy', 'BPSWS1 BGN Curncy', 'BPSWS1F BGN Curncy', 'BPSWS2 BGN Curncy', 'BPSWS3 BGN Curncy', 'BPSWS4 BGN Curncy', 'BPSWS5 BGN Curncy', 'BPSWS6 BGN Curncy', 'BPSWS7 BGN Curncy', 'BPSWS8 BGN Curncy', 'BPSWS9 BGN Curncy', 'BPSWS10 BGN Curncy', 'BPSWS12 BGN Curncy', 'BPSWS15 BGN Curncy', 'BPSWS20 BGN Curncy', 'BPSWS25 BGN Curncy', 'BPSWS30 BGN Curncy', 'BPSWS35 BGN Curncy', 'BPSWS40 BGN Curncy', 'BPSWS50 BGN Curncy']
#GBP_Tenor = ['1 WK', '2 WK', '1 MO', '2 MO', '3 MO', '4 MO', '5 MO', '6 MO', '7 MO', '8 MO', '9 MO', '10 MO', '11 MO', '12 MO', '18 MO', '2 YR', '3 YR', '4 YR', '5 YR', '6 YR', '7 YR', '8 YR', '9 YR', '10 YR', '12 YR', '15 YR', '20 YR', '25 YR', '30 YR', '35 YR', '40 YR', '50 YR']

# CHF_Curncy = ['SFSNT1Z BGN Curncy', 'SFSNT2Z BGN Curncy', 'SFSNTA BGN Curncy', 'SFSNTB BGN Curncy', 'SFSNTC BGN Curncy', 'SFSNTD BGN Curncy', 'SFSNTE BGN Curncy', 'SFSNTF BGN Curncy', 'SFSNTG BGN Curncy', 'SFSNTH BGN Curncy', 'SFSNTI BGN Curncy', 'SFSNTJ BGN Curncy', 'SFSNTK BGN Curncy', 'SFSNT1 BGN Curncy', 'SFSNT1C BGN Curncy', 'SFSNT1F BGN Curncy', 'SFSNT1I BGN Curncy', 'SFSNT2 BGN Curncy', 'SFSNT3 BGN Curncy', 'SFSNT4 BGN Curncy', 'SFSNT5 BGN Curncy', 'SFSNT6 BGN Curncy', 'SFSNT7 BGN Curncy', 'SFSNT8 BGN Curncy', 'SFSNT9 BGN Curncy', 'SFSNT10 BGN Curncy', 'SFSNT12 BGN Curncy', 'SFSNT15 BGN Curncy', 'SFSNT20 BGN Curncy', 'SFSNT25 BGN Curncy', 'SFSNT30 BGN Curncy']
#CHF_Tenor = ['1 WK', '2 W', '1 MO', '2 MO', '3 MO', '4 MO', '5 MO', '6 MO', '7 MO', '8 MO', '9 MO', '10 MO', '11 MO', '12 MO', '15 MO', '18 MO', '21 MO', '2 YR', '3 YR', '4 YR', '5 YR', '6 YR', '7 YR', '8 YR', '9 YR', '10 YR', '12 YR', '15 YR', '20 YR', '25 YR', '30 YR']

import pdblp
from xbbg import blp
con = pdblp.BCon(timeout=2000)
con.start()

Ticker_data=[]
Refresh_Data=[{"strikes_list":"BNPE GR 12/16/22 C52 Equity","ticker_name":"BNP FP"}]
for i in Refresh_Data:
    Ticker_data.append(i['ticker_name'])    
print(Ticker_data[0])

Ticker_value = blp.bdp(Ticker_data[0] + "Equity",flds=['CRNCY'])
Ticker_value=Ticker_value.loc[:,"crncy"]

Search_Bloom_ticker=''
for i in Ticker_value:
    Search_Bloom_ticker=i
    
print(Search_Bloom_ticker)

EURO_Curncy= ['EESWE1Z BGN Curncy', 'EESWE2Z BGN Curncy', 'EESWEA BGN Curncy', 'EESWEB BGN Curncy', 'EESWEC BGN Curncy', 'EESWEF BGN Curncy',  'EESWEI BGN Curncy', 'EESWE1 BGN Curncy', 'EESWE1F BGN Curncy', 'EESWE2 BGN Curncy', 'EESWE3 BGN Curncy'] 
EURO_tenor =['1 WK', '2 WK', '1 MO', '2 MO', '3 MO', '6 MO', '9 MO', '12 MO', '18 MO', '2 YR', '3 YR']
Euro_days= [7, 14, 30, 60, 90, 180, 270, 360, 540, 720, 1080]


GBP_Curncy = ['BPSWS1Z BGN Curncy', 'BPSWS2Z BGN Curncy', 'BPSWSA BGN Curncy', 'BPSWSB BGN Curncy', 'BPSWSC BGN Curncy', 'BPSWSD BGN Curncy', 'BPSWSE BGN Curncy', 'BPSWSG BGN Curncy', 'BPSWSH BGN Curncy', 'BPSWSI BGN Curncy',  'BPSWSJ BGN Curncy', 'BPSWSK BGN Curncy', 'BPSWS1 BGN Curncy', 'BPSWS1F BGN Curncy', 'BPSWS2 BGN Curncy','BPSWS3 BGN Curncy', 'BPSWS4 BGN Curncy']
GBP_Tenor = ['1 WK', '2 WK', '1 MO', '2 MO', '3 MO', '4 MO', '5 MO', '6 MO', '7 MO', '8 MO', '9 MO', '10 MO', '11 MO', '12 MO', '18 MO', '2 YR', '3 YR']
GBP_days= [7, 14, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 540, 720, 1080]

CHF_Curncy = ['SFSNT1Z BGN Curncy', 'SFSNT2Z BGN Curncy', 'SFSNTA BGN Curncy', 'SFSNTB BGN Curncy', 'SFSNTC BGN Curncy','SFSNTD BGN Curncy', 'SFSNTE BGN Curncy', 'SFSNTF BGN Curncy', 'SFSNTG BGN Curncy', 'SFSNTH BGN Curncy','SFSNTI BGN Curncy', 'SFSNTJ BGN Curncy', 'SFSNTK BGN Curncy', 'SFSNT1 BGN Curncy', 'SFSNT1C BGN Curncy','SFSNT1F BGN Curncy', 'SFSNT1I BGN Curncy', 'SFSNT2 BGN Curncy', 'SFSNT3 BGN Curncy']       
CHF_Tenor = ['1 WK', '2 W', '1 MO', '2 MO', '3 MO', '4 MO', '5 MO', '6 MO', '7 MO', '8 MO', '9 MO', '10 MO', '11 MO', '12 MO', '15 MO', '18 MO', '21 MO', '2 YR', '3 YR']
CHF_days= [7, 14, 30, 60, 90, 120, 150, 180, 210, 240, 270, 300, 330, 360, 450, 540, 630, 720, 1080]


Input_value= Search_Bloom_ticker
if(Input_value== 'EUR'):
    Input_value=EURO_Curncy
    Input_tenor=EURO_tenor
    Input_days= Euro_days
elif(Input_value== 'GBp'):
    Input_value=GBP_Curncy
    Input_tenor=GBP_Tenor
    Input_days= GBP_days
elif(Input_value== 'CHF'):
    Input_value=CHF_Curncy
    Input_tenor=CHF_Tenor
    Input_days= CHF_days
else:
    print('No Match Found')
    
#print(Input_value)
bloom_data_array, Final_display_array=[],[]

for i in range(len(Input_value)):
    bloom_data=con.ref(Input_value[i] ,flds=["PX_LAST"])
    bloom_data_array.append(bloom_data['value'])
    
for i in  range(len(bloom_data_array)):
    False_display=[]
    False_display.append(Input_tenor[i])
    False_display.append(Input_days[i])
    False_display.append(bloom_data_array[i][0])
    Final_display_array.insert(i,False_display)

print('Final array :',Final_display_array)


BNP FP
EUR
Final array : [['1 WK', 7, 1.4028], ['2 WK', 14, 1.4035], ['1 MO', 30, 1.5174], ['2 MO', 60, 1.7481], ['3 MO', 90, 1.92], ['6 MO', 180, 2.2738], ['9 MO', 270, 2.4966], ['12 MO', 360, 2.6179], ['18 MO', 540, 2.6745], ['2 YR', 720, 2.654], ['3 YR', 1080, 2.5634]]
